In [1]:
import dataclasses
from typing import Any, Literal

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.axes import Axes
from matplotlib.figure import Figure

from src.drawing_utils import Arrow, Point, Segment
from src.plotting_utils import configure_matplotlib, rm
from scipy.interpolate import interp1d

configure_matplotlib()

In [2]:
@dataclasses.dataclass
class SupplyDemandPlotter:
    ax: Axes
    xlim: tuple[float, float] = (0.0, 10.0)
    ylim: tuple[float, float] = (0.0, 10.0)
    xticks: dict[float, str] | None = None
    yticks: dict[float, str] | None = None
    stepped: bool = True
    equilibrium: Point | None = None

    def __post_init__(self) -> None:
        self.ax.set_aspect("equal")
        self.ax.spines[:].set_visible(False)
        self.ax.set_xlim(self.xlim[0] - 0.3, self.xlim[1] + 1)
        self.ax.set_ylim(self.ylim[0] - 0.3, self.ylim[1] + 1)
        if self.xticks is not None:
            self.ax.set_xticks(list(self.xticks.keys()))
        if self.yticks is not None:
            self.ax.set_yticks(list(self.yticks.keys()))
        if self.xticks is not None:
            self.ax.set_xticklabels(list(self.xticks.values()))
        if self.yticks is not None:
            self.ax.set_yticklabels(list(self.yticks.values()))
        Arrow.horizontal(x1=self.xlim[0], x2=(self.xlim[1] + 1)).drawn(
            self.ax
        ).end.labeled(self.ax, "$Q$", ha="left", va="center")
        Arrow.vertical(y1=self.ylim[0], y2=(self.ylim[1] + 1)).drawn(
            self.ax
        ).end.labeled(self.ax, "$P$", va="bottom")

    def add(
        self,
        supply_or_demand: Literal["supply", "demand"],
        curve: list[Point],
    ) -> None:
        xs, ys = np.array([p.xy for p in curve]).T
        fmt = "o-" if self.stepped else "-"
        color = {"supply": "#1565C0", "demand": "#EF6C00"}[supply_or_demand]
        drawstyle = "steps" if self.stepped else "default"
        self.ax.plot(
            xs,
            ys,
            fmt,
            color=color,
            drawstyle=drawstyle,
            label=rm(f"{supply_or_demand.capitalize()} Curve"),
        )
        x_vals = np.linspace(*self.xlim, 500)
        if self.equilibrium is not None:
            interpolator = interp1d(
                xs,
                ys,
                kind=("next" if self.stepped else "linear"),
                fill_value="extrapolate",
            )
            self.ax.fill_between(
                x_vals,
                interpolator(x_vals),
                0,
                where=(x_vals <= self.equilibrium.x),
                step=("pre" if self.stepped else None),
                alpha=0.2,
                color=color,
                hatch={"supply": r"\\", "demand": "//"}[supply_or_demand],
                edgecolor=color,
                label=rm(
                    {"supply": "Seller Cost, $C$", "demand": "Buyer Utility, $U$"}[
                        supply_or_demand
                    ]
                ),
            )
            self.equilibrium.drawn(self.ax)

    def legend(self) -> None:
        self.ax.legend(loc="center left", bbox_to_anchor=(0.95, 0.5))

In [ ]:
fig, ax = plt.subplots(figsize=(6.4, 4.2), layout="tight")

x_vals = np.linspace(0, 10)
supply_demand = SupplyDemandPlotter(
    ax,
    equilibrium=Point(4.5, 5.5).labeled(
        ax, "$(Q^*, P^*)$", ha="left", va="center", offset=(10, 1)
    ),
    stepped=False,
    xticks={},
    yticks={},
)
supply_demand.add("supply", [Point(x, 1 + 1 * x) for x in x_vals])
supply_demand.add("demand", [Point(x, 10 - x) for x in x_vals])
supply_demand.legend()

fig.savefig("img/fig_2_1.png", dpi=200)

In [ ]:
fig, ax = plt.subplots(figsize=(6.4, 4.2), layout="tight")

eq = Point(6, 5)
supply_demand = SupplyDemandPlotter(ax, equilibrium=Point(6, 5))
supply_demand.add("supply", [Point(0, 0), Point(6, 2), Point(9, 7)])
supply_demand.add("demand", [Point(0, 8), Point(4, 8), Point(8, 5)])
supply_demand.legend()
Segment(eq, (6.5, 4.5)).drawn(ax).end.labeled(ax, "$(Q^*, P^*)$", ha="left", va="top")

fig.savefig("img/fig_2_2.png", dpi=200)

In [5]:
def two_generators_two_loads(mode: Literal["ticks", "area"]) -> Figure:
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, layout="tight")

    def plot(
        ax: Axes,
        supply_or_demand: Literal["supply", "demand"],
        curve: list[Point],
        Q_star: float,
        label: str,
    ):
        kwargs: dict[str, Any] = (
            dict(
                equilibrium=Point(Q_star, np.nan),
                xticks={Q_star: rf"$Q_\mathrm{{{label}}}^*$"},
                yticks={},
            )
            if mode == "area"
            else {}
        )
        SupplyDemandPlotter(ax, **kwargs).add(supply_or_demand, curve)
        ax.set_title(rm(label))

    plot(ax1, "supply", [Point(0, 0), Point(6, 2), Point(9, 7)], 6, "G1")
    if mode == "area":
        Point(3, 1).labeled(ax1, r"$C_\mathrm{G1}(Q_\mathrm{G1}^*)$")
    plot(ax2, "supply", [Point(0, 0), Point(7, 4), Point(10, 10)], 5, "G2")
    if mode == "area":
        Point(2.5, 2).labeled(ax2, r"$C_\mathrm{G2}(Q_\mathrm{G2}^*)$")
    plot(ax3, "demand", [Point(0, 8), Point(4, 8), Point(8, 5)], 8, "L1")
    if mode == "area":
        Point(4, 3.25).labeled(ax3, r"$U_\mathrm{L2}(Q_\mathrm{L2}^*)$")
    plot(ax4, "demand", [Point(0, 9), Point(3, 9), Point(9, 3)], 3, "L2")
    if mode == "area":
        Segment((1.5, 4.5), (4.5, 5)).drawn(ax4).end.labeled(
            ax4, r"$U_\mathrm{L2}(Q_\mathrm{L2}^*)$", ha="left"
        )

    return fig

In [ ]:
fig = two_generators_two_loads(mode="ticks")
fig.savefig("img/fig_2_3.png", dpi=200)

In [ ]:
fig = two_generators_two_loads(mode="area")
fig.savefig("img/fig_2_4.png", dpi=200)

In [8]:
def add_horizontal_brace(
    ax: Axes, x1: float, x2: float, y: float, label: str, opening: Literal["up", "down"]
) -> None:
    center_x = (x2 + x1) / 2
    sign = {"up": -1, "down": 1}[opening]
    ax.annotate(
        label,
        xy=(center_x, y + 0.25 * sign),
        xytext=(center_x, y + 1 * sign),
        ha="center",
        va={"up": "top", "down": "bottom"}[opening],
        arrowprops=dict(arrowstyle=f"-[, widthB={(x2 - x1) / 2 * 1.17 - 0.2}"),
    )

In [ ]:
fig, ax = plt.subplots(figsize=(6.4, 3.6), layout="tight")

eq = Point(11, 4)
supply_demand = SupplyDemandPlotter(ax, xlim=(0, 20), equilibrium=eq)
supply_demand.add(
    "supply", [Point(0, 0), Point(6, 2), Point(13, 4), Point(15, 7), Point(19, 10)]
)
add_horizontal_brace(ax, x1=0, x2=6, y=2, label=r"$Q_\mathrm{G1}^*$", opening="up")
add_horizontal_brace(ax, x1=6, x2=11, y=4, label=r"$Q_\mathrm{G2}^*$", opening="up")
supply_demand.add(
    "demand", [Point(0, 9), Point(3, 9), Point(7, 8), Point(11, 5), Point(17, 3)]
)
add_horizontal_brace(ax, x1=0, x2=3, y=9, label=r"$Q_\mathrm{L1}^*$", opening="down")
add_horizontal_brace(ax, x1=3, x2=11, y=8, label=r"$Q_\mathrm{L2}^*$", opening="down")
supply_demand.legend()
Segment(eq, Point(14, 5)).drawn(ax).end.labeled(ax, "$(Q^*, P^*)$", ha="left")

fig.savefig("img/fig_2_5.png", dpi=200)